In [1]:
import pandas as pd
import numpy as np
import requests
import time
import re
import json
import time
import random

import spotipy
import tweepy

from bs4 import BeautifulSoup
from matplotlib import pyplot as plt
from collections import defaultdict
from collections import deque
from fake_useragent import UserAgent

%matplotlib inline

In [37]:
test = pd.read_csv("helper/test.csv")
train = pd.read_csv("helper/train.csv")
artists = pd.read_csv("helper/artists.csv")
profiles = pd.read_csv("helper/profiles.csv")

In [38]:
median_age = profiles.age.median()
profiles.age = [median_age if (age > 110) or (age < 0) or (pd.isnull(age)) else age for age in profiles.age]

In [35]:
artistdf = pd.read_pickle('helper/artist_df.pkl')
artistdf = artistdf.rename(columns={'artist_id' : 'artist'})
artistdf.shape

(2000, 201)

In [5]:
user_medians = {}
for user in train.groupby('user') :
    user_medians[user[0]] = user[1]['plays'].median()
    
GLOBAL_USER_MEDIAN = np.median(user_medians.values())
print GLOBAL_USER_MEDIAN

106.0


In [6]:
artist_medians = {}
for artist in train.groupby('artist') :
    artist_medians[artist[0]] = artist[1]['plays'].median()
    
GLOBAL_ARTIST_MEDIAN = np.median(artist_medians.values())
print GLOBAL_ARTIST_MEDIAN 

113.0


In [7]:
def add_medians(df, user_medians, artist_medians) :
    df2 = df.copy()
    df2['user_median'] = [user_medians[user] for user in df2['user']]
    df2['artist_median'] = [artist_medians[artist] for artist in df2['artist']]
    return df2

In [8]:
sorted_user_medians = sorted(user_medians.items(), key=lambda x: x[1], reverse=True)
outlier_users = [el[0] for el in sorted_user_medians if el[1] > 1000]
normal_users = [el[0] for el in sorted_user_medians if el[1] <= 1000]

In [59]:
X_train_normal = train[train['user'].isin(normal_users)]
X_train_normal = add_medians(X_train_normal, user_medians, artist_medians)

X_train_outlier = train[train['user'].isin(outlier_users)]
X_train_outlier = add_medians(X_train_outlier, user_medians, artist_medians)

In [60]:
X_test_normal = test[test['user'].isin(normal_users)]
X_test_normal = add_medians(X_test_normal, user_medians, artist_medians)

X_test_outlier = test[test['user'].isin(outlier_users)]
X_test_outlier = add_medians(X_test_outlier, user_medians, artist_medians)

In [39]:
artistdf = artistdf[['artist','started','n_album','is_group','is_male']]
profiles['is_male'] = pd.get_dummies(profiles["sex"])["m"]
profiles['born'] = [2016 - age for age in profiles.age]
profiles = profiles.drop(['country','sex','age'],axis=1)

In [45]:
X_train_normal_users = pd.merge(X_train_normal, profiles, on="user")
X_train_normal_artists = pd.merge(X_train_normal_users, artistdf, on="artist").drop(['user','artist'],axis=1)
X_train_normal_artists['age_diff'] = X_train_normal_artists['started'] - X_train_normal_artists['born']
y_train_normal = X_train_normal_artists['plays']
X_train_normal = X_train_normal_artists.drop(['plays','born','started'],axis=1)

In [48]:
X_train_outlier_users = pd.merge(X_train_outlier, profiles, on="user")
X_train_outlier_artists = pd.merge(X_train_outlier_users, artistdf, on="artist").drop(['user','artist'],axis=1)
X_train_outlier_artists['age_diff'] = X_train_outlier_artists['started'] - X_train_outlier_artists['born']
y_train_outlier = X_train_outlier_artists['plays']
X_train_outlier = X_train_outlier_artists.drop(['plays','born','started'],axis=1)

In [51]:
from sklearn.linear_model import LinearRegression
LR_normal = LinearRegression()
LR_normal.fit(X_train_normal,y_train_normal)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [52]:
LR_outlier = LinearRegression()
LR_outlier.fit(X_train_outlier,y_train_outlier)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [61]:
X_test_normal_users = pd.merge(X_test_normal, profiles, on="user")
X_test_normal_artists = pd.merge(X_test_normal_users, artistdf, on="artist").drop(['user','artist'],axis=1)
X_test_normal_artists['age_diff'] = X_test_normal_artists['started'] - X_test_normal_artists['born']
X_normal_ids = X_test_normal_artists['Id']
X_test_normal = X_test_normal_artists.drop(['Id','born','started'],axis=1)

In [62]:
X_test_outlier_users = pd.merge(X_test_outlier, profiles, on="user")
X_test_outlier_artists = pd.merge(X_test_outlier_users, artistdf, on="artist").drop(['user','artist'],axis=1)
X_test_outlier_artists['age_diff'] = X_test_outlier_artists['started'] - X_test_outlier_artists['born']
X_outlier_ids = X_test_outlier_artists['Id']
X_test_outlier = X_test_outlier_artists.drop(['Id','born','started'],axis=1)

In [65]:
y_normal_pred = LR_normal.predict(X_test_normal)
y_outlier_pred = LR_outlier.predict(X_test_outlier)

In [77]:
y_all_pred = list(y_normal_pred) + list(y_outlier_pred)
ids = list(X_normal_ids) + list(X_outlier_ids)

In [80]:
toExport = pd.DataFrame()
toExport['Id'] = ids
toExport['plays'] = y_all_pred
toExport.sort("Id", inplace=True)
toExport.to_csv('split_model.csv',index=False)

In [81]:
split_model = pd.read_csv('split_model.csv')

In [83]:
user_medians = pd.read_csv('user_median.csv')